In [1]:
@file:DependsOn("ai.timefold.solver:timefold-solver-core:1.14.0")


In [2]:
import ai.timefold.solver.core.api.domain.lookup.PlanningId
import ai.timefold.solver.core.api.domain.entity.PlanningEntity
import ai.timefold.solver.core.api.domain.variable.PlanningVariable
import ai.timefold.solver.core.api.domain.variable.PlanningVariableGraphType
import ai.timefold.solver.core.api.domain.variable.InverseRelationShadowVariable
import ai.timefold.solver.core.api.domain.variable.ShadowVariable
import ai.timefold.solver.core.api.score.buildin.hardsoft.HardSoftScore
import ai.timefold.solver.core.api.domain.solution.PlanningSolution
import ai.timefold.solver.core.api.domain.solution.PlanningEntityCollectionProperty
import ai.timefold.solver.core.api.domain.solution.ProblemFactCollectionProperty
import ai.timefold.solver.core.api.domain.solution.PlanningScore
import ai.timefold.solver.core.api.domain.valuerange.ValueRangeProvider
import ai.timefold.solver.core.api.score.stream.ConstraintProvider
import ai.timefold.solver.core.api.score.stream.ConstraintFactory
import ai.timefold.solver.core.api.score.stream.Constraint
import ai.timefold.solver.core.api.score.stream.Joiners
import ai.timefold.solver.core.api.solver.SolverFactory
import ai.timefold.solver.core.api.solver.Solver
import ai.timefold.solver.core.config.solver.SolverConfig
import ai.timefold.solver.core.api.score.director.ScoreDirector
import ai.timefold.solver.core.api.domain.variable.VariableListener

import java.time.LocalDateTime
import java.time.Duration
import kotlin.random.Random


In [6]:
data class Skill(
    val id: String,
    val name: String
)

data class Location(
    val latitude: Double,
    val longitude: Double
) {
    // Simplified distance calculation (Euclidean)
    fun distanceTo(other: Location): Double {
        val xDiff = latitude - other.latitude
        val yDiff = longitude - other.longitude
        return kotlin.math.sqrt(xDiff * xDiff + yDiff * yDiff)
    }

    // Assuming average speed of 50 km/h for travel time calculation
    fun travelTimeTo(other: Location): Duration {
        val distanceKm = distanceTo(other) * 111  // Approximate conversion to kilometers
        val averageSpeedKmh = 50.0
        val timeInHours = distanceKm / averageSpeedKmh
        return Duration.ofMinutes((timeInHours * 60).toLong())
    }
}

data class TimeWindow(
    val start: LocalDateTime,
    val end: LocalDateTime
)


In [4]:
interface Standstill {
    val location: Location
    val departureTime: LocalDateTime?
}


In [8]:
class Agent(
    @PlanningId
    val id: String,
    val name: String,
    val homeLocation: Location,
    val skills: List<Skill>,
    val maxWorkingHoursPerDay: Duration
) : Standstill {
    override val location: Location
        get() = homeLocation

    override val departureTime: LocalDateTime?
        get() = null  // Agents start their day, so no departure time

    // Shadow variable to get the first meeting
    @InverseRelationShadowVariable(sourceVariableName = "previousStandstill")
    var nextMeeting: Meeting? = null
}


Line_8.jupyter.kts (9:28 - 36) Type of 'location' is not a subtype of the overridden property 'public abstract val location: Line_3_jupyter.Location defined in Line_4_jupyter.Standstill'
Line_8.jupyter.kts (17:22 - 29) Unresolved reference: Meeting